## Post-deployment QA: Reply tool Instrumentation - New VEFU Events 

This task is about ensuring that the newly added events to the [VisualEditorFeatureUse](https://meta.wikimedia.org/wiki/Schema:VisualEditorFeatureUse) schema are being logged correctly in database.

Test scenarios and status are documented in this [QC document](https://docs.google.com/spreadsheets/d/1txtypYKZHCiZEFnkye9LX7riF3VEippMmvN6ILs_BUQ/edit#gid=605234531)

In [24]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
    library(tidyverse); library(glue); library(lubridate); library(scales)
})

In [41]:
# Collect VEFU events after deployment
query <- 
"SELECT
    date_format(dt, 'yyyy-MM-dd') AS date,
    wiki AS wiki,
    event.editor_interface AS editor_interface,
    event.platform AS platform,
    event.feature As feature,
    event.integration AS integration,
    event.action AS action,
    COUNT(*) AS events
FROM event.visualeditorfeatureuse
WHERE 
-- reviewing a few days prior to confirm when events started firing
    year = 2020 
    AND month >= 09
    AND wiki <> 'testwiki'
GROUP BY
    date_format(dt, 'yyyy-MM-dd'),
    wiki,
    event.editor_interface,
    event.feature,
    event.integration,
    event.platform,
    event.action
"

In [42]:
collect_vefu_events <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [43]:
collect_vefu_events$date <- as.Date(collect_vefu_events$date, format = "%Y-%m-%d")

# 1. Advanced Affordance Events

Event added that logs when people "open" the Reply Tool's advanced "shelf". 

This functionality can be accessed by clicking the Advanced link that will appear beneath the text input area and can be used to customize the edit summary that accompanies the comments they post with the Reply Tool.

It was deployed on 17 September 2020 to all wikis where the reply tool was accessible. 

**Event**	**Name**
* An event that fires when the Advanced link/affordance is clicked to "show" the Comment summary field	feature: dtReply, action: advanced-show
* An event that fires when the Advanced link/affordance is clicked to "hide" the Comment summary field	feature: dtReply, action: advanced-hide


[Task](https://phabricator.wikimedia.org/T261816)

## Confirm Events Firing as Expected

In [6]:
advanced_show_events <- collect_vefu_events %>%
    filter(action == 'advanced-show') %>%
    arrange(date)

advanced_show_events 

date,wiki,editor_interface,platform,feature,integration,action,events
<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
2020-09-17,arwiki,visualeditor,desktop,dtReply,discussiontools,advanced-show,1
2020-09-17,hewiki,wikitext,desktop,dtReply,discussiontools,advanced-show,1
2020-09-18,frwiki,wikitext,desktop,dtReply,discussiontools,advanced-show,1
2020-09-18,kowiki,wikitext,desktop,dtReply,discussiontools,advanced-show,1
2020-09-18,jawiki,wikitext,desktop,dtReply,discussiontools,advanced-show,1
2020-09-18,zhwiki,wikitext,desktop,dtReply,discussiontools,advanced-show,1
2020-09-19,enwiki,wikitext,desktop,dtReply,discussiontools,advanced-show,1
2020-09-19,jawiki,wikitext,desktop,dtReply,discussiontools,advanced-show,1
2020-09-20,huwiki,visualeditor,desktop,dtReply,discussiontools,advanced-show,1


Events are being recorded as early as 17 September 2020, which was the date of deployment. 

Confirmed that advanced show events are only being recorded for dtreply feature and discussiontools integration as expected. Events are recorded for both wikitext and visualeditor.

In [7]:
advanced_close_events <- collect_vefu_events %>%
    filter(action == 'advanced-hide') %>%
    arrange(date)

advanced_close_events 

date,wiki,editor_interface,platform,feature,integration,action,events
<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
2020-09-17,arwiki,visualeditor,desktop,dtReply,discussiontools,advanced-hide,1
2020-09-17,hewiki,wikitext,desktop,dtReply,discussiontools,advanced-hide,1
2020-09-18,jawiki,wikitext,desktop,dtReply,discussiontools,advanced-hide,1
2020-09-18,zhwiki,wikitext,desktop,dtReply,discussiontools,advanced-hide,1
2020-09-19,jawiki,wikitext,desktop,dtReply,discussiontools,advanced-hide,1
2020-09-20,huwiki,wikitext,desktop,dtReply,discussiontools,advanced-hide,2
2020-09-21,arwiki,wikitext,desktop,dtReply,discussiontools,advanced-hide,1
2020-09-28,svwiki,wikitext,desktop,dtReply,discussiontools,advanced-hide,5
2020-09-29,enwiki,visualeditor,desktop,dtReply,discussiontools,advanced-hide,6


Confirmed there are fewer advanced hide events compared to advanced show events as expected. Advanced hide events also start coming in on 17 September 2020 (date of deployment).

Confirmed that advanced hide events are only being recorded for dtreply feature and discussiontools integration as expected. Events are recorded for both wikitext and visualeditor.

## Total Events

Review total events to confirm there are fewer advanced-hide events to advanced-show events.

In [8]:
total_advanced_events <- collect_vefu_events %>%
    filter(action == 'advanced-hide' |
          action == 'advanced-show') %>%
    group_by(action)  %>%
    summarise(total_events = sum(events))

total_advanced_events 

`summarise()` ungrouping output (override with `.groups` argument)



action,total_events
<chr>,<int>
advanced-hide,71
advanced-show,142


Confirmed that there are more advanced show events compared to advanced hide. As of 26 Oct 2020, there have been 142 advanced-show events and 71 advanced-hide events (213 advanced affordance events total)

There are no events in any other platform other than "Desktop" since this feature is only available on desktop.

## Event Counts by Wiki

In [9]:
total_advanced_events_bywiki <- collect_vefu_events %>%
    filter(action == 'advanced-hide' |
          action == 'advanced-show') %>%
    group_by(action, wiki)  %>%
    summarise(total_events = sum(events)) %>%
    arrange(wiki)

total_advanced_events_bywiki

`summarise()` regrouping output by 'action' (override with `.groups` argument)



action,wiki,total_events
<chr>,<chr>,<int>
advanced-hide,arwiki,7
advanced-show,arwiki,14
advanced-hide,cswiki,2
advanced-show,cswiki,3
advanced-hide,enwiki,6
advanced-show,enwiki,7
advanced-hide,enwikiversity,1
advanced-show,enwikiversity,1
advanced-hide,eswiki,1


Advanced events recorded for arwiki, cswiki, enwiki, frwiki, hewiki, huwiki, jawiki, kowiki, nlwiki, svwiki, viwiki, and zhwiki. A few observations:
* For all wikis, there are more advanced-show events then advanced-hide events as expected.
* For viwiki and kowiki, there is only one advanced-show event recorded (no advanced-hide event).
* Enwiki and HeWiki have events but are not listed on the deployment wikis; however, these are likely wikis where it was enabled for testing using ?dtenable=1.


In [10]:
total_advanced_events_bywiki_both <- collect_vefu_events %>%
    filter(action == 'advanced-hide' |
          action == 'advanced-show') %>%
    group_by( wiki)  %>%
    summarise(total_events = sum(events)) %>%
    arrange(total_events)

total_advanced_events_bywiki_both 

`summarise()` ungrouping output (override with `.groups` argument)



wiki,total_events
<chr>,<int>
metawiki,1
viwiki,1
enwikiversity,2
hewiki,2
itwiki,2
kawiki,2
plwiki,2
ptwiki,2
eswiki,3


Svwiki and Nlwiki have the largest number of advanced affordance events recorded to date (55 for svwiki and 32 for nlwiki).

# 2. Dialog Prevent Show Event 
Event added that is fired when someone attempts to switch from the Reply Tool's source to visual mode after drafting a comment that contains any combination of the following syntax: extension, template or table.

This is associated with the editor_switch feature and discussiontools integration.

[Task](https://phabricator.wikimedia.org/T259673)

## Confirm events firing as expected

In [44]:
switch_prevent_events <- collect_vefu_events %>%
    filter(action == 'dialog-prevent-show') %>%
    arrange(desc(date))

switch_prevent_events

date,wiki,editor_interface,platform,feature,integration,action,events
<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
2020-11-10,nlwiki,wikitext,desktop,editor-switch,discussiontools,dialog-prevent-show,1
2020-11-09,nlwiki,visualeditor,desktop,editor-switch,discussiontools,dialog-prevent-show,2
2020-11-03,arwiki,visualeditor,desktop,editor-switch,discussiontools,dialog-prevent-show,1
2020-10-30,plwiki,visualeditor,desktop,editor-switch,discussiontools,dialog-prevent-show,1
2020-10-25,arwiki,visualeditor,desktop,editor-switch,discussiontools,dialog-prevent-show,1
2020-10-16,arwiki,visualeditor,desktop,editor-switch,discussiontools,dialog-prevent-show,1
2020-10-14,frwiki,visualeditor,desktop,editor-switch,discussiontools,dialog-prevent-show,1
2020-10-13,nlwiki,visualeditor,desktop,editor-switch,discussiontools,dialog-prevent-show,2


Since the deployment date, only 7 dialog-prevent-show events have been recorded. Many of these are likely testing events. It appears that this dialog appears very rarely based on the current data. 

All events are for the editor_switch feature and discussion tools integration as expected. The editing interface for these events are recorded as 'visual editor' - Is that correct given that this action is supposed to record when an editor is prevented from switching to visual mode from source?

Logged an event on Fr.Wiki, cswiki, and kowiki. May take some time for event to come in. 